# 📚 מדריך מקיף: טכניקות טרנספורמציה של נתונים ב-Polars

**ברוכים הבאים למדריך המקיף ביותר לעבודה עם Polars!**

מחברת זו תלווה אתכם צעד אחר צעד בלמידת כל הכלים החשובים לעיבוד וניתוח נתונים.

---

## 📋 תוכן עניינים

1. [הקדמה והגדרת סביבה](#הקדמה)
2. [צברים פשוטים (Simple Aggregations)](#צברים-פשוטים)
   - [מהם צברים?](#מהם-צברים)
   - [פעולות צבירה בסיסיות](#פעולות-צבירה-בסיסיות)
   - [צבירה על טורים ספציפיים](#צבירה-על-טורים-ספציפיים)
   - [תרגילים - צברים פשוטים](#תרגילים-צברים-פשוטים)
3. [קיבוץ וצבירה (Group By Aggregations)](#קיבוץ-וצבירה)
   - [הבנת group_by](#הבנת-group_by)
   - [צבירה מרובת טורים](#צבירה-מרובת-טורים)
   - [קיבוץ לפי מספר עמודות](#קיבוץ-לפי-מספר-עמודות)
   - [תרגילים - קיבוץ וצבירה](#תרגילים-קיבוץ-וצבירה)
4. [צבירה אופקית (Horizontal Aggregations)](#צבירה-אופקית)
   - [sum_horizontal ו-mean_horizontal](#sum_horizontal)
   - [שימוש ב-concat_list](#concat_list)
   - [reduce ו-fold](#reduce-fold)
   - [תרגילים - צבירה אופקית](#תרגילים-צבירה-אופקית)
5. [פונקציות חלון (Window Functions)](#פונקציות-חלון)
   - [מהן פונקציות חלון?](#מהן-פונקציות-חלון)
   - [שימוש ב-over()](#over)
   - [ranking וסידור](#ranking)
   - [תרגילים - פונקציות חלון](#תרגילים-פונקציות-חלון)
6. [פונקציות מוגדרות משתמש (UDFs)](#udfs)
   - [מתי להשתמש ב-UDFs](#מתי-udfs)
   - [map_elements](#map_elements)
   - [אלטרנטיבות מובנות](#אלטרנטיבות-מובנות)
   - [תרגילים - UDFs](#תרגילים-udfs)
7. [SQL ב-Polars](#sql)
   - [שילוב SQL וPolars](#שילוב-sql)
   - [שאילתות מתקדמות](#שאילתות-מתקדמות)
   - [תרגילים - SQL](#תרגילים-sql)
8. [סיכום ומשאבים נוספים](#סיכום)

---

<a id='הקדמה'></a>
## 🚀 הקדמה והגדרת סביבה

### מה זה Polars?

**Polars** היא ספריית Python מודרנית ומהירה במיוחד לעבודה עם נתונים טבלאיים (DataFrames). היא:

- ⚡ **מהירה מאוד** - כתובה ב-Rust ומנצלת מעבדים מרובי ליבות
- 🧠 **חסכונית בזיכרון** - משתמשת ב-Apache Arrow
- 🔗 **אקספרסיבית** - תחביר נקי וקריא
- 📊 **עוצמתית** - תומכת בפעולות מורכבות

### התקנה

```bash
pip install polars
```

In [ ]:
# ייבוא הספרייה
import polars as pl

# בדיקת גרסה
print(f"Polars version: {pl.__version__}")

### 📁 טעינת נתונים לדוגמה

נשתמש בקובץ `contoso_sales.csv` - מערך נתונים של מכירות.

In [ ]:
# טעינת הנתונים
df = pl.read_csv('../data/contoso_sales.csv', try_parse_dates=True)

# הצגת 5 השורות הראשונות
df.head()

In [ ]:
# מידע על המבנה
print(f"מספר שורות: {df.height}")
print(f"מספר עמודות: {df.width}")
print(f"\nשמות העמודות: {df.columns}")

In [ ]:
# סקירת טיפוסי הנתונים
df.schema

> 💡 **טיפ חשוב**: הפרמטר `try_parse_dates=True` מנסה לזהות אוטומטית עמודות תאריך ולהמיר אותן לטיפוס Date.

---
<a id='צברים-פשוטים'></a>
## 📊 1. צברים פשוטים (Simple Aggregations)

<a id='מהם-צברים'></a>
### מהם צברים?

**צבירה (Aggregation)** היא פעולה שמקבלת מספר ערכים ומחזירה ערך יחיד.

**דוגמאות:**
- `sum()` - סכום
- `mean()` - ממוצע
- `min()` / `max()` - מינימום/מקסימום
- `count()` - ספירה
- `first()` / `last()` - ערך ראשון/אחרון

<a id='פעולות-צבירה-בסיסיות'></a>
### פעולות צבירה בסיסיות

In [ ]:
# צבירה על כל העמודות המספריות
from polars import selectors as cs

(
    df
    .select(cs.numeric())  # בחירת עמודות מספריות בלבד
    .sum()                 # סכום של כל עמודה
)

**הסבר שלב אחר שלב:**

1. `cs.numeric()` - selector שבוחר את כל העמודות מטיפוס מספרי
2. `.select()` - בוחר רק את העמודות האלה
3. `.sum()` - מבצע סכימה על כל עמודה

**תוצאה:** DataFrame עם שורה אחת המכילה את הסכומים של כל העמודות המספריות.

<a id='צבירה-על-טורים-ספציפיים'></a>
### צבירה על טור ספציפי

In [ ]:
# שיטה 1: המרה ל-Series ואז צבירה
s = df.select('Quantity').to_series()
print(f"סך הכמות (Series): {s.sum()}")

In [ ]:
# שיטה 2: שימוש ב-pl.col() (מומלץ!)
df.select(pl.col('Quantity').sum())

> ✅ **Best Practice**: השתמש ב-`pl.col()` כשאתה עובד עם expressions - זה יותר גמיש ואקספרסיבי.

### צבירות נוספות

In [ ]:
# שימוש ב-first() ו-last()
df.select(
    pl.col('Customer Name').first().alias('שם לקוח ראשון'),
    pl.col('Customer Name').last().alias('שם לקוח אחרון')
)

**הסבר:**
- `first()` מחזיר את הערך הראשון בעמודה
- `last()` מחזיר את הערך האחרון
- `alias()` נותן שם חדש לעמודה בתוצאה

In [ ]:
# פונקציית describe() - סטטיסטיקה תיאורית
df.select(cs.numeric()).describe()

**מה מקבלים מ-describe()?**

- `count` - מספר הערכים
- `null_count` - מספר ערכים חסרים
- `mean` - ממוצע
- `std` - סטיית תקן
- `min` - מינימום
- `25%`, `50%`, `75%` - רבעונים (percentiles)
- `max` - מקסימום

### 🎯 צבירה מותנית

לפעמים נרצה לצבור רק ערכים שעומדים בתנאי מסוים.

In [ ]:
# ספירת כמה הזמנות עם כמות >= 4
df.select(
    (pl.col('Quantity') >= 4).sum().alias('הזמנות עם כמות 4+')
)

**איך זה עובד?**

1. `pl.col('Quantity') >= 4` יוצר עמודת boolean (True/False)
2. `sum()` על boolean סופר כמה True יש (True=1, False=0)

זו דרך חכמה לספור כמה ערכים עומדים בתנאי!

In [ ]:
# סכום כמות רק עבור החנות המקוונת
df.select(
    pl.col('Quantity')
      .filter(pl.col('Store Name') == 'Online store')
      .sum()
      .alias('סך כמות בחנות מקוונת')
)

**הסבר:**
- `filter()` מסנן את הערכים לפני הצבירה
- רק שורות שעומדות בתנאי יילקחו בחשבון בסכימה

<a id='תרגילים-צברים-פשוטים'></a>
### 📝 תרגילים - צברים פשוטים

**תרגיל 1:** חשב את הממוצע של עמודת 'Unit Price'

In [ ]:
# הקוד שלך כאן


**תרגיל 2:** מצא את המחיר המקסימלי והמינימלי בעמודת 'Net Price'

In [ ]:
# הקוד שלך כאן


**תרגיל 3:** ספור כמה לקוחות מארה"ב ('United States') יש במערך הנתונים  
רמז: השתמש ב-filter() + count() או בשיטת הבוליאנים

In [ ]:
# הקוד שלך כאן


---
<a id='קיבוץ-וצבירה'></a>
## 👥 2. קיבוץ וצבירה (Group By Aggregations)

<a id='הבנת-group_by'></a>
### הבנת group_by

**קיבוץ (Grouping)** מאפשר לנו לחלק את הנתונים לקבוצות ולבצע צבירה על כל קבוצה בנפרד.

**תהליך:**
1. **Split** - חלוקה לקבוצות לפי ערכים בעמודה
2. **Apply** - ביצוע פעולה על כל קבוצה
3. **Combine** - איחוד התוצאות

In [ ]:
# יצירת אובייקט GroupBy
grouped = df.group_by('Brand')
print(type(grouped))
print(grouped)

> 📌 **שימו לב:** `group_by()` לבד לא מחשב כלום - הוא רק מכין את הקיבוץ. צריך להוסיף `agg()` כדי לבצע צבירה.

In [ ]:
# קיבוץ + צבירה
(
    df
    .group_by('Brand')
    .agg(pl.col('Quantity').sum().alias('סך כמות'))
    .head()
)

**מה קורה כאן?**

1. כל השורות מתחלקות לקבוצות לפי ערך ב-'Brand'
2. עבור כל קבוצה, מחושב סכום ה-'Quantity'
3. התוצאה: שורה אחת לכל מותג עם הסכום הכולל

<a id='צבירה-מרובת-טורים'></a>
### צבירה מרובת טורים

In [ ]:
# מספר צבירות על אותה קבוצה
(
    df
    .group_by('Brand')
    .agg(
        pl.col('Unit Price').sum().alias('סכום מחירים'),
        pl.col('Unit Price').mean().alias('ממוצע מחיר'),
    )
    .sort('ממוצע מחיר', descending=True)
    .head()
)

**טיפ מתקדם:** אפשר לבצע חישובים מורכבים בתוך `agg()`:

In [ ]:
(
    df
    .group_by('Brand')
    .agg(
        pl.col('Unit Price').mean().round(2).alias('ממוצע מחיר'),
        (pl.col('Unit Price').sum() / pl.len()).round(2).alias('ממוצע מחיר 2'),
        pl.col('Customer Name').first().alias('לקוח ראשון'),
        pl.col('Category').last().alias('קטגוריה אחרונה')
    )
    .sort('Brand')
    .head()
)

**הסברים:**
- `pl.len()` - מספר השורות בקבוצה
- `round(2)` - עיגול ל-2 ספרות אחרי הנקודה
- שתי הדרכים לחישוב ממוצע נותנות תוצאה זהה!

### ⚡ Lazy Evaluation

**Lazy mode** מאפשר ל-Polars לבצע אופטימיזציה לפני ההרצה.

In [ ]:
# שימוש ב-scan_csv במקום read_csv
(
    pl.scan_csv('../data/contoso_sales.csv', try_parse_dates=True)  # Lazy
    .group_by('Brand')
    .agg(
        pl.col('Unit Price').mean().round(2).alias('ממוצע מחיר'),
    )
    .sort('Brand')
    .collect()  # כאן מתבצעת ההרצה בפועל
    .head()
)

> 💡 **מתי להשתמש ב-Lazy?**
> - עבודה עם קבצים גדולים
> - שרשראות פעולות מורכבות
> - כשרוצים לאפשר אופטימיזציה אוטומטית

### 🎨 הגדרות תצוגה

לפעמים מחרוזות ארוכות נחתכות בתצוגה. נוכל לשנות את ההגדרות:

In [ ]:
# שיטה 1: דרך משתני סביבה
import os
os.environ['POLARS_FMT_STR_LEN'] = str(50)

df.select('Brand').unique().head(10)

<a id='קיבוץ-לפי-מספר-עמודות'></a>
### קיבוץ לפי מספר עמודות

In [ ]:
# איטרציה על קבוצות
for name, data in df.group_by(['Brand']):
    print(f"מותג: {name[0]}, טיפוס: {type(data)}")
    break  # נעצור אחרי קבוצה אחת

In [ ]:
# קיבוץ בלי צבירה - רק איסוף הערכים
(
    df
    .group_by('Brand')
    .agg(pl.col('Quantity'))  # ללא פונקציית צבירה!
    .head()
)

**שימו לב:** כשלא משתמשים בפונקציית צבירה, מקבלים רשימות (lists) של כל הערכים בקבוצה.

In [ ]:
# שמירת סדר המקור
(
    df
    .group_by('Brand', maintain_order=True)  # שומר על הסדר!
    .agg(pl.col('Quantity'))
    .head()
)

> ⚠️ **חשוב:** כברירת מחדל, `group_by()` לא שומר על סדר המקור. אם הסדר חשוב, הוסף `maintain_order=True`.

In [ ]:
# קיבוץ לפי מספר עמודות + חישובים
(
    df
    .group_by(
        pl.col('Brand'), 
        'Customer Country',
        pl.col('Order Date').dt.year().alias('שנת הזמנה')  # חילוץ שנה מתאריך
    )
    .agg(pl.col('Unit Price').mean().alias('ממוצע מחיר'))
    .head()
)

**נקודות מפתח:**
- אפשר לערבב שמות עמודות ו-expressions
- `.dt.year()` מחלץ את השנה מעמודת תאריך
- הקיבוץ יוצר קבוצה נפרדת לכל שילוב ייחודי של הערכים

<a id='תרגילים-קיבוץ-וצבירה'></a>
### 📝 תרגילים - קיבוץ וצבירה

**תרגיל 1:** קבץ לפי 'Category' וחשב את סך ה-'Quantity' לכל קטגוריה. מיין בסדר יורד.

In [ ]:
# הקוד שלך כאן


**תרגיל 2:** מצא את הממוצע והחציון של 'Customer Age' לכל 'Customer Gender'

In [ ]:
# הקוד שלך כאן
# רמז: השתמש ב-median() לחציון


**תרגיל 3:** קבץ לפי 'Category' ו-'Subcategory' וחשב כמה הזמנות (שורות) יש בכל קבוצה

In [ ]:
# הקוד שלך כאן
# רמז: pl.len() או count()


---
<a id='צבירה-אופקית'></a>
## ↔️ 3. צבירה אופקית על מספר עמודות

### מה ההבדל בין צבירה אנכית לאופקית?

- **צבירה אנכית** (Vertical): פעולה על ערכים באותה עמודה (למשל: סכום כל המכירות)
- **צבירה אופקית** (Horizontal): פעולה על מספר עמודות באותה שורה (למשל: סכום נקודות של פוקימון)

### טעינת נתונים חדשים

In [ ]:
# נתוני פוקימון
pokemon_df = pl.read_csv('../data/pokemon.csv')
pokemon_df.head()

<a id='sum_horizontal'></a>
### sum_horizontal ו-mean_horizontal

In [ ]:
# חישוב סך הנקודות לכל פוקימון
stat_columns = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

(
    pokemon_df
    .select(stat_columns)
    .sum_horizontal()
    .alias('סך נקודות')
    .head()
)

**הסבר:** `sum_horizontal()` סוכם את הערכים בכל השורות לרוחב (בכל העמודות שנבחרו).

In [ ]:
# הוספת העמודה למערך המקורי
(
    pokemon_df
    .with_columns(
        pl.sum_horizontal('HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed')
          .alias('סך נקודות')
    )
    .head()
)

> 💡 **שימו לב:** `with_columns()` מוסיף עמודות חדשות מבלי להסיר את הקיימות.

<a id='concat_list'></a>
### שימוש ב-concat_list

In [ ]:
# גישה אלטרנטיבית: יצירת רשימה ואז סכימה
(
    pokemon_df
    .with_columns(
        pl.concat_list('HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed')
          .list.sum()
          .alias('סך נקודות')
    )
    .head()
)

**איך זה עובד?**

1. `concat_list()` יוצר עמודת רשימות
2. `.list.sum()` מסכם כל רשימה

**דוגמה:** `[45, 60, 48, 65, 65, 43]` → `326`

<a id='reduce-fold'></a>
### reduce ו-fold - פונקציות מתקדמות

In [ ]:
# שימוש ב-reduce
cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

(
    pokemon_df
    .with_columns(
        pl.reduce(
            function=lambda acc, col: acc + col,  # פונקציה לביצוע
            exprs=pl.col(cols)                     # עמודות להפעיל עליהן
        )
        .alias('סך נקודות')
    )
    .head()
)

**הבדל בין reduce ל-fold:**

- **reduce**: מתחיל מהעמודה הראשונה
- **fold**: מאפשר להגדיר ערך התחלתי (accumulator)

In [ ]:
# שימוש ב-fold עם ערך התחלתי
(
    pokemon_df
    .with_columns(
        pl.fold(
            acc=pl.lit(100),                       # ערך התחלתי = 100
            function=lambda acc, col: acc + col, 
            exprs=pl.col(cols)
        )
        .alias('סך נקודות + 100')
    )
    .head()
)

**למה להשתמש ב-fold?**
- כשצריך ערך התחלתי
- חישובים מורכבים יותר
- שרשור תנאים (כמו AND או OR)

### שימושים מתקדמים

In [ ]:
# סינון: מצא פוקימונים שכל הסטטיסטיקות שלהם מעל 80
(
    pokemon_df
    .filter(
        pl.fold(
            acc=pl.lit(True), 
            function=lambda acc, col: acc & col,  # AND לוגי
            exprs=pl.col(cols) > 80
        )
    )
    .head()
)

**דרך קצרה יותר:**

In [ ]:
# שימוש ב-all_horizontal
(
    pokemon_df
    .filter(
        pl.all_horizontal(pl.col(cols) > 80)
    )
    .head()
)

> ✅ **Best Practice**: השתמש ב-`all_horizontal()` או `any_horizontal()` לתנאים בוליאניים - זה יותר קריא!

In [ ]:
# שרשור מחרוזות
str_cols = ['Name', 'Type 1', 'Type 2']

pokemon_df.select(
    *str_cols,
    pl.concat_str(str_cols, separator=' - ').alias('שרשור')
).head()

<a id='תרגילים-צבירה-אופקית'></a>
### 📝 תרגילים - צבירה אופקית

**תרגיל 1:** חשב את הממוצע האופקי של כל הסטטיסטיקות לכל פוקימון

In [ ]:
# הקוד שלך כאן
# רמז: השתמש ב-mean_horizontal()


**תרגיל 2:** מצא פוקימונים שלפחות אחת מהסטטיסטיקות שלהם מעל 100

In [ ]:
# הקוד שלך כאן
# רמז: השתמש ב-any_horizontal()


**תרגיל 3:** חשב את ההפרש בין הסטטיסטיקה המקסימלית למינימלית לכל פוקימון

In [ ]:
# הקוד שלך כאן
# רמז: השתמש ב-max_horizontal() ו-min_horizontal()


---
<a id='פונקציות-חלון'></a>
## 🪟 4. פונקציות חלון (Window Functions)

<a id='מהן-פונקציות-חלון'></a>
### מהן פונקציות חלון?

**Window Functions** מאפשרות לבצע חישובים על קבוצות כמו `group_by`, אבל **שומרות על כל השורות**.

**ההבדל:**
- `group_by`: מחזיר שורה אחת לקבוצה
- `over()`: שומר את כל השורות + מוסיף מידע מהקבוצה

In [ ]:
# חזרה לנתוני המכירות
df = pl.read_csv('../data/contoso_sales.csv', try_parse_dates=True)

# הוספת עמודת סכום מכירה
df = df.with_columns(
    (pl.col('Quantity') * pl.col('Net Price')).round(2).alias('Sales Amount')
)
df.head()

<a id='over'></a>
### שימוש ב-over()

In [ ]:
# סכום מכירות לפי קטגוריה - לכל שורה!
sales_by_cat = df.select(
    'Category',
    'Subcategory',
    pl.col('Sales Amount').sum().over('Category').alias('סכום מכירות לפי קטגוריה')
)
sales_by_cat.head()

**מה קורה כאן?**

1. עבור כל שורה, Polars מזהה את הקטגוריה
2. מחשב את סכום המכירות של **כל** השורות באותה קטגוריה
3. מוסיף את הערך הזה לשורה הנוכחית

**תוצאה:** כל שורה יודעת מה סך המכירות בקטגוריה שלה!

In [ ]:
# נבדוק עבור קטגוריה אחת
sales_by_cat.filter(pl.col('Category') == 'Audio').unique().head()

In [ ]:
# ההבדל בגדלים:
print(f"DataFrame מקורי: {df.shape}")
print(f"DataFrame עם over(): {sales_by_cat.shape}")
# אותו מספר שורות!

### over() עם מספר עמודות

In [ ]:
# ממוצע מכירות לפי קטגוריה ומותג
(
    df
    .select(
        'Category',
        'Brand',
        'Subcategory',
        pl.col('Sales Amount')
          .mean()
          .over('Category', 'Brand')
          .alias('ממוצע מכירות לפי קטגוריה ומותג')
    )
    .filter(pl.col('Category') == 'Computers')
    .unique()
    .sort('Brand')
    .head(10)
)

### חישובים מורכבים ב-over()

In [ ]:
# חישוב לפי גיל לידה של הלקוח
from datetime import date

curr_yr = date.today().year
cust_birth_yr = curr_yr - pl.col('Customer Age')

(
    df
    .select(
        'Category',
        'Brand',
        'Customer Age',
        pl.col('Sales Amount')
          .mean()
          .over('Category', cust_birth_yr)
          .alias('ממוצע מכירות לפי קטגוריה ושנת לידה')
    )
    .filter(pl.col('Category') == 'Computers')
    .unique()
    .sort('Customer Age')
    .head(10)
)

> 💡 **טיפ מתקדם:** אפשר להשתמש ב-expressions מורכבים בתוך `over()`, לא רק שמות עמודות!

<a id='ranking'></a>
### Ranking וסידור

In [ ]:
# דירוג קטגוריות לפי מכירות מקסימליות
(
    df
    .group_by('Category')
    .agg(pl.col('Sales Amount').max().alias('מכירה מקסימלית'))
    .with_columns(
        pl.col('מכירה מקסימלית').rank(descending=True).alias('דירוג')
    )
    .sort('דירוג')
)

**שימוש ב-rank() עם over():**

In [ ]:
# דירוג תת-קטגוריות בתוך כל קטגוריה
(
    df
    .group_by('Category', 'Subcategory')
    .agg(pl.col('Sales Amount').max().round().cast(pl.Int64).alias('מכירה מקסימלית'))
    .with_columns(
        pl.col('מכירה מקסימלית')
          .rank(descending=True)
          .over('Category')
          .cast(pl.Int64)
          .alias('דירוג בקטגוריה')
    )
    .filter(pl.col('Category').is_in(['Audio', 'Computers']))
    .sort(['Category', 'דירוג בקטגוריה'])
)

**הסבר:**
- הדירוג מחושב **בתוך כל קטגוריה בנפרד**
- כל קטגוריה מקבלת דירוג 1, 2, 3... משלה

### אסטרטגיות Mapping

לפונקציות חלון יש שני מצבי עבודה:

In [ ]:
# הכנת נתונים
max_sales_rank = (
    df
    .group_by('Category', 'Subcategory')
    .agg(pl.col('Sales Amount').max().round().cast(pl.Int64).alias('מכירה מקסימלית'))
    .with_columns(
        pl.col('מכירה מקסימלית')
          .rank(descending=True)
          .over('Category')
          .cast(pl.Int64)
          .alias('דירוג')
    )
    .filter(pl.col('Category').is_in(['Audio', 'Computers']))
    .sort(['Category', 'דירוג'])
)

In [ ]:
# mapping_strategy='join' - שומר רק שורות שמתאימות
max_sales_rank.with_columns(
    pl.col('Subcategory')
      .sort_by('מכירה מקסימלית')
      .head(3)
      .over('Category', mapping_strategy='join')
      .alias('3 תתי-קטגוריות נמוכות')
)

In [ ]:
# mapping_strategy='explode' - משכפל שורות
max_sales_rank.with_columns(
    pl.col('Subcategory')
      .sort_by('מכירה מקסימלית')
      .over('Category', mapping_strategy='explode')
      .alias('תת-קטגוריה ממוינת')
)

> ⚠️ **שימו לב:** `explode` יכול ליצור הרבה שורות - השתמשו בזהירות!

<a id='תרגילים-פונקציות-חלון'></a>
### 📝 תרגילים - פונקציות חלון

**תרגיל 1:** הוסף עמודה שמראה את הכמות הממוצעת לפי מדינת לקוח

In [ ]:
# הקוד שלך כאן


**תרגיל 2:** חשב את אחוז המכירה מסך המכירות בקטגוריה שלה

In [ ]:
# הקוד שלך כאן
# רמז: (Sales Amount / sum().over('Category')) * 100


**תרגיל 3:** דרג מותגים לפי סכום מכירות בתוך כל מדינה

In [ ]:
# הקוד שלך כאן


---
<a id='udfs'></a>
## 🔧 5. פונקציות מוגדרות משתמש (UDFs)

<a id='מתי-udfs'></a>
### מתי להשתמש ב-UDFs?

**UDF (User Defined Function)** = פונקציה שאנחנו כותבים בעצמנו.

**מתי להשתמש?**
- כשאין פונקציה מובנית שעושה מה שצריך
- לוגיקה עסקית מורכבת
- כשמהירות לא קריטית

**⚠️ אזהרה:** UDFs איטיים יותר מפונקציות מובנות!

<a id='map_elements'></a>
### map_elements

הפונקציה `map_elements()` מאפשרת להפעיל פונקציה Python על כל אלמנט.

In [ ]:
# דוגמה 1: חילוץ שם פרטי
def get_first_name(full_name: str) -> str:
    """מחלץ את השם הפרטי משם מלא"""
    return full_name.split(' ')[0]

df.select(
    'Customer Name',
    pl.col('Customer Name')
      .map_elements(lambda el: get_first_name(el), return_dtype=pl.String)
      .alias('שם פרטי')
).head()

**פרמטרים חשובים:**
- `lambda el: ...` - הפונקציה שתופעל
- `return_dtype` - **חובה!** צריך להגדיר מה טיפוס התוצאה

In [ ]:
# דרך קצרה יותר - ישירות עם lambda
df.select(
    'Customer Name',
    pl.col('Customer Name')
      .map_elements(lambda el: el.split(' ')[0], return_dtype=pl.String)
      .alias('שם פרטי')
).head()

### דוגמה מורכבת יותר

In [ ]:
# המרת גיל לטווח
def age_to_range(age: int) -> str:
    """ממיר גיל לטווח גילאים"""
    if age < 18:
        return 'עד 17'
    elif age <= 30:
        return '18-30'
    elif age <= 50:
        return '31-50'
    elif age <= 70:
        return '51-70'
    else:
        return '71+'

df.select(
    'Customer Age',
    pl.col('Customer Age')
      .map_elements(lambda el: age_to_range(el), return_dtype=pl.String)
      .alias('טווח גיל')
).head()

<a id='אלטרנטיבות-מובנות'></a>
### אלטרנטיבות מובנות (מומלץ!)

**כמעט תמיד יש דרך טובה יותר מ-UDF!**

In [ ]:
# חילוץ שם פרטי - בלי UDF!
df.select(
    'Customer Name',
    pl.col('Customer Name')
      .str.split(' ')
      .list.first()
      .alias('שם פרטי')
).head()

**יתרונות של הגישה המובנית:**
- ⚡ הרבה יותר מהירה
- 🔗 עובדת עם lazy evaluation
- 🛠️ מותאמת לאופטימיזציה

In [ ]:
# טווח גילאים - עם when/then (מומלץ!)
df.select(
    'Customer Age',
    pl.when(pl.col('Customer Age') < 18).then(pl.lit('עד 17'))
      .when(pl.col('Customer Age') <= 30).then(pl.lit('18-30'))
      .when(pl.col('Customer Age') <= 50).then(pl.lit('31-50'))
      .when(pl.col('Customer Age') <= 70).then(pl.lit('51-70'))
      .when(pl.col('Customer Age') > 70).then(pl.lit('71+'))
      .alias('טווח גיל')
).head()

### השוואת ביצועים

In [ ]:
%%timeit
# עם UDF
df.select(
    pl.col('Customer Name')
      .map_elements(lambda el: el.split(' ')[0], return_dtype=pl.String)
).head()

In [ ]:
%%timeit
# ללא UDF
df.select(
    pl.col('Customer Name').str.split(' ').list.first()
).head()

> 🚀 **תוצאה:** הפונקציות המובנות מהירות פי כמה וכמה!

<a id='תרגילים-udfs'></a>
### 📝 תרגילים - UDFs

**תרגיל 1:** כתוב UDF שממיר טמפרטורה מצלזיוס לפרנהייט  
נוסחה: F = (C × 9/5) + 32

In [ ]:
# צור DataFrame לדוגמה
temp_df = pl.DataFrame({'celsius': [0, 10, 20, 30, 100]})

# הקוד שלך כאן


**תרגיל 2:** כתוב את אותו חישוב בלי UDF (עם expressions רגילות)

In [ ]:
# הקוד שלך כאן


**תרגיל 3:** כתוב פונקציה שמסווגת מחיר כ-'זול', 'בינוני', או 'יקר'  
זול: < 20, בינוני: 20-100, יקר: > 100  
נסה גם עם when/then!

In [ ]:
# הקוד שלך כאן


---
<a id='sql'></a>
## 🗄️ 6. SQL ב-Polars

<a id='שילוב-sql'></a>
### שילוב SQL וPolars

**Polars תומך ב-SQL!** אפשר לכתוב שאילתות SQL סטנדרטיות.

In [ ]:
# יצירת SQLContext
ctx = pl.SQLContext(eager=True)
ctx.register('df', df)  # רישום ה-DataFrame

# שאילתה פשוטה
ctx.execute(
    """
    SELECT
        `Customer Name`,
        Brand,
        Category
    FROM df
    LIMIT 5
    """
)

**הערות חשובות:**
- שמות עמודות עם רווחים צריכים backticks: `` `Customer Name` ``
- `eager=True` מריץ מיד; `eager=False` מחזיר LazyFrame

<a id='שאילתות-מתקדמות'></a>
### שאילתות מתקדמות

In [ ]:
# GROUP BY + סידור
ctx.execute(
    """
    SELECT
        Brand,
        AVG(Quantity) as avg_quantity
    FROM df
    GROUP BY Brand
    ORDER BY avg_quantity DESC
    LIMIT 5
    """
)

### Lazy SQL

In [ ]:
# שימוש ב-LazyFrame
pl.SQLContext(lf=df.lazy()).execute(
    """
    SELECT 
        Brand,
        Category
    FROM lf
    LIMIT 5
    """
).collect()

> 💡 **מתי להשתמש ב-SQL?**
> - כשאתה יודע SQL טוב יותר מ-Polars
> - כשצריך להמיר קוד SQL קיים
> - לשאילתות מורכבות שקל יותר לכתוב ב-SQL

<a id='תרגילים-sql'></a>
### 📝 תרגילים - SQL

**תרגיל 1:** כתוב שאילת SQL שמחזירה את 10 הלקוחות הצעירים ביותר

In [ ]:
# הקוד שלך כאן


**תרגיל 2:** ספור כמה הזמנות יש לכל מדינה, ממוין בסדר יורד

In [ ]:
# הקוד שלך כאן


**תרגיל 3:** מצא את הקטגוריה עם המחיר הממוצע הגבוה ביותר

In [ ]:
# הקוד שלך כאן


---
<a id='סיכום'></a>
## 🎓 סיכום ומשאבים נוספים

### מה למדנו?

✅ **צברים פשוטים**: sum(), mean(), first(), last()  
✅ **קיבוץ**: group_by() + agg()  
✅ **צבירה אופקית**: sum_horizontal(), reduce(), fold()  
✅ **פונקציות חלון**: over() לחישובים מתקדמים  
✅ **UDFs**: map_elements() (אבל זהירות - איטי!)  
✅ **SQL**: SQLContext לשאילתות SQL

### עצות חשובות

1. **השתמש בפונקציות מובנות** במקום UDFs
2. **Lazy evaluation** לקבצים גדולים
3. **over() > group_by()** כשצריך לשמור שורות
4. **תכנן את השאילתות** לפני שכותבים קוד

### 📚 משאבים נוספים

- [תיעוד רשמי של Polars](https://pola-rs.github.io/polars/)
- [Polars Cookbook](https://pola-rs.github.io/polars-book/)
- [Polars על GitHub](https://github.com/pola-rs/polars)

### 🚀 המשך לימוד

- עבודה עם נתונים גדולים (Streaming)
- אופטימיזציה וביצועים
- שילוב עם Arrow ו-Parquet
- עבודה עם סדרות זמן

---

## 🎉 כל הכבוד!

סיימת את המדריך המקיף לטרנספורמציה של נתונים ב-Polars!  
עכשיו אתה מוכן להתחיל לעבוד עם נתונים בצורה מקצועית ויעילה.

**בהצלחה! 🚀**